# Postprocessing

Makes control plots and templates.

Authors: Raghav Kansal, Cristina Suarez

In [ ]:
import os
import pickle
from HH4b import utils, plotting, postprocessing
from HH4b.postprocessing import Region, weight_shifts
from HH4b.utils import ShapeVar, CUT_MAX_VAL
from HH4b.hh_vars import samples, data_key, bg_keys, sig_keys, LUMI

from pathlib import Path
from hist import Hist

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

from copy import deepcopy

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = "../../../"
# this is the directory to the files
# path_to_dir = f"{MAIN_DIR}/../data/skimmer/Oct26/"
path_to_dir = f"{MAIN_DIR}/../data/skimmer/23Nov10_v12/"
year = "2022"

# make plot and template directory
date = "23Nov14_v12"
plot_dir = Path(f"{MAIN_DIR}/plots/PostProcessing/{date}/{year}")
template_dir = f"templates/{date}/"
_ = os.system(f"mkdir -p {plot_dir}")
_ = os.system(f"mkdir -p {template_dir}/cutflows/{year}")

In [ ]:
# define dictionary with directories of files (this can be configured in a yaml file later in the script)
samples = deepcopy(samples[year])
sig_keys = ["hh4b"]
for key in list(samples.keys()):
    if key not in bg_keys + sig_keys + [data_key]:
        del samples[key]

sample_dirs = {path_to_dir: samples}

In [ ]:
# columns to load
# the parquet files are too big so we can only load a few columns at a time without consumming much memory
load_columns = [
    ("weight", 1),
    # ("DijetMass", 1),
    ("ak8FatJetPt", 2),
    ("ak8FatJetPNetXbb", 2),
    ("ak8FatJetMsd", 2),
    ("ak8FatJetPNetMass", 2),
    ("ht", 1),
    ("nPV", 1),
    # ("single_weight_pileup", 1),
]

if "v12" in path_to_dir:
    load_columns += [("ak8FatJetPNetMassRaw", 2)]

load_columns_mc = load_columns + [
    ("single_weight_pileup", 1),
    ("single_weight_genWeight"),
    ("single_weight_trigsf_2jet"),
]
# + [(f"weight_{syst}_{shift}", 1) for syst in weight_shifts for shift in ["up", "down"]]

In [ ]:
# pd.read_parquet(f"{path_to_dir}/2022EE/QCD_HT-2000/parquet").columns
# pd.read_parquet(f"{path_to_dir}/2022EE/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_TSG/parquet").columns
list(
    pd.read_parquet(
        f"{path_to_dir}/2022/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_TSG/parquet"
    ).columns
)

In [ ]:
hhcf = utils.get_cutflow(
    f"{path_to_dir}/2022EE/GluGlutoHHto4B_kl-0p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_TSG/pickles",
    year,
    "GluGlutoHHto4B_kl-0p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_TSG",
)
hhcf

In [ ]:
pt_cut = 300
msd_cut = 40

filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", pt_cut),
        ("('ak8FatJetPt', '1')", ">=", pt_cut),
        ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        ("('ak8FatJetMsd', '1')", ">=", msd_cut),
        ("('ak8FatJetPNetXbb', '0')", ">=", 0.8),
    ],
    [
        ("('ak8FatJetPt', '0')", ">=", pt_cut),
        ("('ak8FatJetPt', '1')", ">=", pt_cut),
        ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        ("('ak8FatJetMsd', '1')", ">=", msd_cut),
        ("('ak8FatJetPNetXbb', '1')", ">=", 0.8),
    ],
]

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()))

# dictionary that will contain all information (from all samples)
events_dict = {}
for input_dir, samples in sample_dirs.items():
    events_dict = {
        **events_dict,
        # this function will load files (only the columns selected), apply filters and compute a weight per event
        **utils.load_samples(
            input_dir,
            samples,
            year,
            filters=filters,
            columns=utils.format_columns(load_columns),
            # columns_mc=utils.format_columns(load_columns_mc),
        ),
    }

utils.add_to_cutflow(events_dict, "Preselection", "weight", cutflow)
print("\n", cutflow)

### Weights

Removing high QCD event weight

In [ ]:
# plt.hist(events_dict["qcd"]["weight"][events_dict["qcd"]["ak8FatJetPt"][0] > 1250], bins=np.power(10, np.linspace(-4, 5, 100)))
plt.hist(
    events_dict["qcd"]["single_weight_genweight"][events_dict["qcd"]["ak8FatJetPt"][0] > 300],
    bins=np.power(10, np.linspace(-4, 7, 100)),
)
# plt.hist(events_dict["qcd"]["single_weight_genweight"], bins=np.power(10, np.linspace(-4, 7, 100)))
plt.hist(events_dict["qcd"]["weight"], bins=np.power(10, np.linspace(-4, 5, 100)))
plt.yscale("log")
plt.xscale("log")
plt.show()

In [ ]:
events_dict["qcd"] = events_dict["qcd"][(events_dict["qcd"]["weight"] < 1e2).values]
# remove super high gen weight events for high pt qcd
events_dict["qcd"] = events_dict["qcd"][
    ~(
        (events_dict["qcd"]["ak8FatJetPt"][0] > 400)
        & (events_dict["qcd"]["single_weight_genweight"][0] > 1e6)
    ).values
]

Checking pileup weights and removing from signal

In [ ]:
fig = plt.figure(figsize=(12, 12))
plt.rcParams.update({"font.size": 24})
plt.hist(
    events_dict["hh4b"]["single_weight_pileup"],
    np.linspace(0, 10, 21),
    histtype="step",
    label="HH4b",
)
plt.hist(
    events_dict["qcd"]["single_weight_pileup"], np.linspace(0, 10, 21), histtype="step", label="QCD"
)
plt.hist(
    events_dict["ttbar"]["single_weight_pileup"],
    np.linspace(0, 10, 21),
    histtype="step",
    label="TT",
)
plt.legend()
plt.yscale("log")
plt.xlabel("Pileup Weights")
plt.ylabel("Events")
hep.cms.label(data=False, year=year, lumi=round(LUMI[year] / 1e3))
plt.savefig(f"{plot_dir}/pileup_weights.pdf", bbox_inches="tight")
plt.show()

In [ ]:
if "Oct26" in path_to_dir:
    events_dict["hh4b"]["weight"] /= events_dict["hh4b"]["single_weight_pileup"]
utils.add_to_cutflow(events_dict, "Fix Weights", "weight", cutflow)
cutflow

### Control Plots

In [ ]:
ylims = {
    "2022": 5e4,
    "2022EE": 4e5,
}

# {var: (bins, label)}
control_plot_vars = [
    # var must match key in events dictionary (i.e. as saved in parquet file)
    # ShapeVar(var="DijetMass", label=r"$m^{jj}$ (GeV)", bins=[30, 600, 4000]),
    ShapeVar(var="ak8FatJetPt0", label=r"$p_T^{j1}$ (GeV)", bins=[50, 300, 1500]),
    # ShapeVar(var="ak8FatJetPt1", label=r"$p_T^{j2}$ (GeV)", bins=[50, 300, 1500]),
    # ShapeVar(
    #     var="ak8FatJetPNetMass0",
    #     label=r"$m_{reg}^{j1}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetPNetMass1",
    #     label=r"$m_{reg}^{j2}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetMsd0",
    #     label=r"$m_{SD}^{j1}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetMsd1",
    #     label=r"$m_{SD}^{j2}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetPNetXbb0",
    #     label=r"$T_{Xbb}^{j1}$",
    #     bins=[50, 0.0, 1],
    # ),
    # ShapeVar(
    #     var="ak8FatJetPNetXbb1",
    #     label=r"$T_{Xbb}^{j2}$",
    #     bins=[50, 0.0, 1],
    # ),
    # ShapeVar(
    #     var="ht",
    #     label=r"HT (GeV)",
    #     bins=[50, 0, 4000],
    # ),
]

hists = {}
for shape_var in control_plot_vars:
    if shape_var.var not in hists:
        hists[shape_var.var] = utils.singleVarHist(
            events_dict,
            shape_var,
            weight_key="weight",
        )

for shape_var in control_plot_vars:
    name = f"{plot_dir}/{shape_var.var}.pdf"
    plotting.ratioHistPlot(
        hists[shape_var.var],
        year,
        sig_keys,
        bg_keys,
        name=name,
        show=True,
        log=True,
        plot_significance=False,
        significance_dir=shape_var.significance_dir,
        ratio_ylims=[0.2, 1.8],
        ylim=ylims[year],
    )
    # break

mSD vs mReg

In [ ]:
mplot_dir = Path(plot_dir) / "msd_mreg"
mplot_dir.mkdir(exist_ok=True)

plot_events = {
    "hh4b": "HH4b",
    "qcd": "QCD",
    "ttbar": "TT",
    "data": "Data",
}

bins_hh4b = np.linspace(30, 250, 23)
bins_all = np.linspace(30, 330, 31)

mass_vars = {}

for ev_key in plot_events:
    bins = bins_hh4b if ev_key == "hh4b" else bins_all
    mass_vars[ev_key] = [
        ShapeVar(
            var="ak8FatJetPNetMass",
            label="Regressed (JEC-Corrected)" if "v12" in path_to_dir else "Regressed",
            bins=bins,
            reg=False,
            plot_args={"color": plotting.colours["lightred"]},
        ),
        ShapeVar(
            var="ak8FatJetMsd",
            label="Soft Drop",
            bins=bins,
            reg=False,
            plot_args={"color": plotting.colours["darkblue"]},
        ),
    ]

    if "v12" in path_to_dir:
        mass_vars[ev_key].append(
            ShapeVar(
                var="ak8FatJetPNetMassRaw",
                label="Regressed (Raw)",
                bins=bins,
                reg=False,
                plot_args={"color": plotting.colours["orange"]},
            )
        )

hists = {}

for ev_key, ev_label in plot_events.items():
    events = events_dict[ev_key]
    hists[ev_key] = {}

    for i in range(2):
        for mvar in mass_vars[ev_key]:
            hists[ev_key][f"{mvar.var}{i}"] = h = Hist(mvar.axis, storage="weight")
            h.fill(events[mvar.var][i], weight=events["weight"])

            if ev_key == "hh4b":
                hists[ev_key][f"{mvar.var}{i}_mean"] = np.mean(events[mvar.var][i])
                hists[ev_key][f"{mvar.var}{i}_std"] = np.std(events[mvar.var][i])

with mplot_dir.joinpath("hists.pkl").open("wb") as f:
    pickle.dump(hists, f)

In [ ]:
mass_vars_v11 = [
    ShapeVar(
        var="ak8FatJetMsd",
        label="Soft Drop",
        plot_args={"color": plotting.colours["darkblue"]},
    ),
    ShapeVar(
        var="ak8FatJetPNetMass",
        label="Regressed v11",
        plot_args={"color": plotting.colours["lightred"]},
    ),
]

mass_vars_v12 = [
    ShapeVar(
        var="ak8FatJetPNetMass",
        label="Regressed v12",
        plot_args={"color": plotting.colours["orange"]},
    ),
    # ShapeVar(
    #     var="ak8FatJetPNetMassRaw",
    #     label="Regressed v12 (Raw)",
    #     plot_args={"color": plotting.colours["forestgreen"]},
    # ),
]

with plot_dir.parent.parent.joinpath("23Nov14_v11/2022EE/msd_mreg/hists.pkl").open("rb") as f:
    hists_v11 = pickle.load(f)

with plot_dir.parent.parent.joinpath("23Nov14_v12/2022/msd_mreg/hists.pkl").open("rb") as f:
    hists_v12 = pickle.load(f)

for ev_key, ev_label in plot_events.items():
    for i in range(2):
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        plt.rcParams.update({"font.size": 24})

        for mvar in mass_vars_v11:
            h = hists_v11[ev_key][f"{mvar.var}{i}"]
            if ev_key == "hh4b":
                add_label = f"\n$\\overline{{m}} = {hists_v11[ev_key][f'{mvar.var}{i}_mean']:.1f}, \sigma_m = {hists_v11[ev_key][f'{mvar.var}{i}_std']:.1f}$ GeV"
            else:
                add_label = ""

            hep.histplot(
                h,
                histtype="step",
                label=f"{mvar.label}{add_label}",
                linewidth=2,
                yerr=False,
                density=True,
                **mvar.plot_args,
            )

        for mvar in mass_vars_v12:
            h = hists_v12[ev_key][f"{mvar.var}{i}"]
            if ev_key == "hh4b":
                add_label = f"\n$\\overline{{m}} = {hists_v12[ev_key][f'{mvar.var}{i}_mean']:.1f}, \sigma_m = {hists_v12[ev_key][f'{mvar.var}{i}_std']:.1f}$ GeV"
            else:
                add_label = ""

            hep.histplot(
                h,
                histtype="step",
                label=f"{mvar.label}{add_label}",
                linewidth=2,
                yerr=False,
                density=True,
                **mvar.plot_args,
            )

        ax.set_title(ev_label, x=0.42, y=1.002)
        ax.set_xlabel(f"AK8 Jet {i + 1} Mass")
        ax.set_ylabel("Events (A.U.)")
        ax.set_ylim(0)

        # hep.cms.label(
        #     data=True, label="Internal", year=year, lumi=round(LUMI[year] / 1e3), com="13.6"
        # )

        hep.cms.label(
            data=True,
            label="Internal",
            rlabel="",
        )

        ax.legend(fontsize=18)
        plt.savefig(mplot_dir / f"{ev_key}_j{i}_msd_mreg_noraw.pdf", bbox_inches="tight")
        plt.show()

    #     break
    # break

In [ ]:
mplot_dir = Path(plot_dir) / "msd_mreg"
mplot_dir.mkdir(exist_ok=True)

plot_events = {
    "hh4b": "HH4b",
    "qcd": "QCD",
    "ttbar": "TT",
    "data": "Data",
}

bins = np.linspace(30, 250, 23)

mass_vars = [
    ShapeVar(
        var="ak8FatJetPNetMass",
        label="Regressed (JEC-Corrected)" if "v12" in path_to_dir else "Regressed",
        bins=bins,
        reg=False,
        plot_args={"color": plotting.colours["lightred"]},
    ),
    ShapeVar(
        var="ak8FatJetMsd",
        label="Soft Drop",
        bins=bins,
        reg=False,
        plot_args={"color": plotting.colours["darkblue"]},
    ),
]

if "v12" in path_to_dir:
    mass_vars.append(
        ShapeVar(
            var="ak8FatJetPNetMassRaw",
            label="Regressed (Raw)",
            bins=bins,
            reg=False,
            plot_args={"color": plotting.colours["orange"]},
        )
    )

for ev_key, ev_label in plot_events.items():
    events = events_dict[ev_key]

    for i in range(2):
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        plt.rcParams.update({"font.size": 24})

        bins = np.linspace(0, 250, 26)

        for mvar in mass_vars:
            if ev_key == "hh4b":
                mean = np.mean(events[mvar.var][i])
                std = np.std(events[mvar.var][i])
                add_label = f"\n$\\overline{{m}} = {mean:.1f}, \sigma_m = {std:.1f}$ GeV"
            else:
                add_label = ""

            ax.hist(
                events[mvar.var][i],
                bins=mvar.bins if ev_key == "hh4b" else np.linspace(30, 330, 31),
                histtype="step",
                label=f"{mvar.label}{add_label}",
                linewidth=2,
                # alpha=0.5,
                # hatch="//",
                **mvar.plot_args,
            )

        ax.set_title(ev_label, x=0.42, y=1.002)
        ax.set_xlabel(f"AK8 Jet {i + 1} Mass")
        ax.set_ylabel("Events")

        hep.cms.label(
            data=True, label="Internal", year=year, lumi=round(LUMI[year] / 1e3), com="13.6"
        )

        ax.legend(fontsize=18)
        plt.savefig(mplot_dir / f"{ev_key}_j{i}_msd_mreg.pdf", bbox_inches="tight")
        plt.show()

    #     break
    # break

Pileup and trigger weight checks

In [ ]:
for key, events in events_dict.items():
    if key == data_key:
        events["weight_notrig"] = events["weight"]
    else:
        events["weight_notrig"] = events["weight"] / events["single_weight_trigsf_2jet"]

In [ ]:
for key, events in events_dict.items():
    if key in [data_key] + sig_keys:
        events["weight_nopileup"] = events["weight"]
    else:
        events["weight_nopileup"] = events["weight"] / events["single_weight_pileup"]

In [ ]:
# {var: (bins, label)}
control_plot_vars = [
    # var must match key in events dictionary (i.e. as saved in parquet file)
    # ShapeVar(var="DijetMass", label=r"$m^{jj}$ (GeV)", bins=[30, 600, 4000]),
    ShapeVar(var="ak8FatJetPt0", label=r"$p_T^{j1}$ (GeV)", bins=[50, 300, 1500]),
    # ShapeVar(var="ak8FatJetPt0", label=r"$p_T^{j1}$ (GeV)", bins=[20, 300, 400]),
    # ShapeVar(var="ak8FatJetPt1", label=r"$p_T^{j2}$ (GeV)", bins=[50, 300, 1500]),
    # ShapeVar(
    #     var="ak8FatJetPNetMass0",
    #     label=r"$m_{reg}^{j1}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetPNetMass1",
    #     label=r"$m_{reg}^{j2}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetMsd0",
    #     label=r"$m_{SD}^{j1}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetMsd1",
    #     label=r"$m_{SD}^{j2}$ (GeV)",
    #     bins=[19, 60, 250],
    #     significance_dir="bin",
    # ),
    # ShapeVar(
    #     var="ak8FatJetPNetXbb0",
    #     label=r"$T_{Xbb}^{j1}$",
    #     bins=[50, 0.0, 1],
    # ),
    # ShapeVar(
    #     var="ak8FatJetPNetXbb1",
    #     label=r"$T_{Xbb}^{j2}$",
    #     bins=[50, 0.0, 1],
    # ),
    # ShapeVar(var="nPV", label=r"nPV", bins=[20, 0, 80])
    # ShapeVar(
    #     var="ht",
    #     label=r"HT (GeV)",
    #     bins=[50, 0, 4000],
    # ),
]

# for weight_key in ["weight", "weight_nopileup"]:
for weight_key in ["weight_nopileup"]:
    # for weight_key in ["weight", "weight_notrig"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_dict,
                shape_var,
                weight_key=weight_key,
            )

    for shape_var in control_plot_vars:
        name = f"{plot_dir}/{shape_var.var}_{weight_key}.pdf"
        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sig_keys,
            bg_keys,
            name=name,
            show=True,
            log=True,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
            # title="With Pileup Reweighting"
            # if weight_key == "weight"
            # else "Without Pileup Reweighting",
        )
        # break

In [ ]:
# signal only
events = events_dict["hh4b"]
bins = np.arange(60, 201, 8)
plt.figure(figsize=(12, 12))
plt.hist(
    events["ak8FatJetMsd"][0],
    bins=bins,
    label="No Correction",
    # histtype="step",
    weights=events["weight_notrig"],
    color=plotting.colours["slategray"],
    linewidth=2,
    alpha=0.5,
    hatch="//",
)
plt.hist(
    events["ak8FatJetMsd"][0],
    bins=bins,
    label="With Corrections",
    histtype="step",
    weights=events["weight"],
    color=plotting.colours["orange"],
    linewidth=2,
)
plt.legend()
hep.cms.label(data=False, year=year, lumi=round(LUMI[year] / 1e3), com="13.6")
plt.title("HH4b", x=0.45, y=1.005)
plt.ylabel("Events")
plt.xlabel(r"Jet 1 $m_{SD}$(GeV)")
plt.ylim(0)
plt.savefig(f"{plot_dir}/msd_trig_weights.pdf", bbox_inches="tight")
plt.show()

Mass plots for signal

In [ ]:
sig_key = "hh4b"
plots = {
    "bb0FatJetPNetMass": r"$m_{reg}^{j1}$ (GeV)",
    "bb1FatJetPNetMass": r"$m_{reg}^{j2}$ (GeV)",
    "bb0FatJetMsd": r"$m_{SD}^{j1}$ (GeV)",
    "bb1FatJetMsd": r"$m_{SD}^{j2}$ (GeV)",
}

for key, label in plots.items():
    plt.hist(
        utils.get_feat(events_dict[sig_key], key),
        bins=20,
        histtype="step",
        label=label,
        range=[50, 250],
        weights=utils.get_feat(events_dict[sig_key], "weight"),
        linestyle="--",
        linewidth=2,
    )

plt.legend()
plt.xlabel("Mass (GeV)")
plt.ylabel("Events")
plt.show()

Control plots without trigger weights

In [ ]:
hists_notrigweights = {}
for shape_var in control_plot_vars:
    if shape_var.var not in hists_notrigweights:
        hists_notrigweights[shape_var.var] = utils.singleVarHist(
            events_dict,
            shape_var,
            weight_key="weight_notrig",
        )

for shape_var in control_plot_vars:
    name = f"{plot_dir}/{shape_var.var}_notrigweights.pdf"
    plotting.ratioHistPlot(
        hists_notrigweights[shape_var.var],
        year,
        sig_keys,
        bg_keys,
        name=name,
        show=True,
        log=True,
        ratio_ylims=None,
    )

Pt plots for different Txbb cuts

In [ ]:
# {var: (bins, label)}
control_plot_vars = [
    # var must match key in events dictionary (i.e. as saved in parquet file)
    # ShapeVar(var="DijetMass", label=r"$m^{jj}$ (GeV)", bins=[30, 600, 4000]),
    ShapeVar(var="ak8FatJetPt0", label=r"$p_T^{j1}$ (GeV)", bins=[50, 300, 1500]),
    ShapeVar(var="ak8FatJetPt1", label=r"$p_T^{j2}$ (GeV)", bins=[50, 300, 1500]),
    ShapeVar(var="ak8FatJetMsd0", label=r"$m_{SD}^{j1}$ (GeV)", bins=[19, 60, 250]),
    ShapeVar(var="ak8FatJetMsd1", label=r"$m_{SD}^{j2}$ (GeV)", bins=[19, 60, 250]),
]

for txbb_cut in [0, 0.35, 0.6, 0.8, 0.9, 0.95, 0.985]:
    print(txbb_cut)
    hists = {}
    sel, _ = utils.make_selection(
        {"ak8FatJetPNetXbb0+ak8FatJetPNetXbb1": [txbb_cut, CUT_MAX_VAL]}, events_dict
    )
    print(np.mean(sel["qcd"]))
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_dict,
                shape_var,
                weight_key="weight",
                selection=sel,
            )

    for shape_var in control_plot_vars:
        name = f"{plot_dir}/{shape_var.var}_txbb{txbb_cut}.pdf"
        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sig_keys,
            bg_keys,
            name=name,
            show=True,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            title=rf"At least one Jet's $T_{{Xbb}}$ > {txbb_cut}",
        )
        # break

ROC Curve

In [ ]:
sig_jets_score = events_dict["gghtobb"]["ak8FatJetPNetXbb"].values
sig_jets_score = np.max(sig_jets_score, axis=1)
# # copy array values to second column
# sig_jets_score = np.stack((sig_jets_score, sig_jets_score), axis=1)

bg_jets_score = events_dict["qcd"]["ak8FatJetPNetXbb"].values.reshape(-1)

In [ ]:
from sklearn.metrics import roc_curve


bg_skip = 4
sig_key = "gghtobb"
bg_keys = ["qcd"]  # add ttbar too?

y_true = np.concatenate(
    [
        np.ones(len(sig_jets_score)),
        np.zeros(int(np.ceil(len(bg_jets_score) / bg_skip))),
    ]
)

weights = np.concatenate(
    [events_dict[sig_key]["weight"].values.reshape(-1)]
    + [
        np.stack(
            (events_dict[bg_key]["weight"].values, events_dict[bg_key]["weight"].values), axis=1
        ).reshape(-1)[::bg_skip]
        for bg_key in bg_keys
    ],
)

scores = np.concatenate((sig_jets_score, bg_jets_score[::bg_skip]))

fpr, tpr, thresholds = roc_curve(y_true, scores, sample_weight=weights)

In [ ]:
roc = {
    "fpr": fpr,
    "tpr": tpr,
    "thresholds": thresholds,
}

with open(f"{plot_dir}/roc.pkl", "wb") as f:
    pickle.dump(roc, f)

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
# plot_thresholds = [0.35, 0.8, 0.95, 0.975, 0.985, 0.99]  # v11
plot_thresholds = [0.1, 0.35, 0.8, 0.9, 0.94, 0.96]  # v12
th_colours = ["#9381FF", "#1f78b4", "#a6cee3", "#ff7f00", "#7CB518", "#EDB458", "#36213E"]

pths = {th: [[], []] for th in plot_thresholds}
plt.figure(figsize=(12, 12))
plt.plot(
    tpr,
    fpr,
    linewidth=2,
)

for th in plot_thresholds:
    idx = find_nearest(thresholds, th)
    pths[th][0].append(tpr[idx])
    pths[th][1].append(fpr[idx])


for k, th in enumerate(plot_thresholds):
    plt.scatter(
        *pths[th],
        marker="o",
        s=40,
        label=rf"$T_{{Xbb}}$ > {th}",
        color=th_colours[k],
        zorder=100,
    )

    plt.vlines(
        x=pths[th][0],
        ymin=0,
        ymax=pths[th][1],
        color=th_colours[k],
        linestyles="dashed",
        alpha=0.5,
    )

    plt.hlines(
        y=pths[th][1],
        xmin=0,
        xmax=pths[th][0],
        color=th_colours[k],
        linestyles="dashed",
        alpha=0.5,
    )


hep.cms.label(data=False, year=year, com="13.6")
# plt.hlines(y=0.01, xmin=0, xmax=1, colors="lightgrey", linestyles="dashed")
plt.yscale("log")
plt.xlabel("Signal efficiency")
plt.ylabel("Background efficiency")
# plt.suptitle(f"HVV FatJet {pvars['title']} ROC", y=0.95)
# plt.title(cut_labels[cutstr], fontsize=20)
plt.xlim([0, 1])
plt.ylim([1e-3, 1])
plt.legend(loc="upper left")
plt.savefig(f"{plot_dir}/roccurve.pdf", bbox_inches="tight")

In [ ]:
pths

### Cut-based selection

In [ ]:
bb_masks = postprocessing.bb_assignment(events_dict)

In [ ]:
# {label: {cutvar: [min, max], ...}, ...}
txbb_cut = 0.985
pt_cut = 300

selection_regions = {
    "pass": Region(
        cuts={
            "bb0FatJetPNetXbb": [txbb_cut, CUT_MAX_VAL],
            "bb1FatJetPNetXbb": [txbb_cut, CUT_MAX_VAL],
            "bb0FatJetPt": [pt_cut, CUT_MAX_VAL],
            "bb1FatJetPt": [pt_cut, CUT_MAX_VAL],
            "bb0FatJetPNetMass": [100, 150],
        },
        label="Pass",
    ),
    "fail": Region(
        cuts={
            "bb0FatJetPNetXbb": [-CUT_MAX_VAL, txbb_cut],
            "bb1FatJetPNetXbb": [-CUT_MAX_VAL, txbb_cut],
            "bb0FatJetPt": [pt_cut, CUT_MAX_VAL],
            "bb1FatJetPt": [pt_cut, CUT_MAX_VAL],
            "bb0FatJetPNetMass": [100, 150],
        },
        label="Fail",
    ),
}

fit_shape_var = ShapeVar(
    "bb1FatJetPNetMass",
    r"$m^{2}_\mathrm{Reg}$ (GeV)",
    [19, 60, 250],
    reg=True,
    blind_window=[100, 150],
)

In [ ]:
templates = postprocessing.get_templates(
    events_dict,
    bb_masks,
    year,
    sig_keys,
    selection_regions,
    [fit_shape_var],
    {},
    template_dir,
    bg_keys,
    plot_dir,
    cutflow,
    weight_key="weight",
    # weight_shifts=weight_shifts,
    show=True,
)

In [ ]:
templates

In [ ]:
postprocessing.save_templates(
    templates,
    f"{template_dir}/{year}_templates.pkl",
    fit_shape_var,
)

In [ ]:
import pickle

with open(f"templates/23Oct17/{year}_templates.pkl", "rb") as f:
    templates2 = pickle.load(f)